<a href="https://colab.research.google.com/github/sharathshanmukh/Recommendations-systems/blob/main/collaborative_filtering_using_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd

In [2]:
import zipfile
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip --no-check-certificate

--2021-05-24 17:44:05--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  11.4MB/s    in 0.4s    

2021-05-24 17:44:06 (11.4 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [3]:

zipfile.ZipFile("/content/ml-100k.zip").extractall()

In [13]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df=pd.read_csv("/content/ml-100k/u.data",sep='\t',names=column_names).drop("timestamp",axis=1)

In [15]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1').rename(columns={"movie_id":"item_id"})

In [24]:
movies_df=movies[["title","item_id"]]
movies_df[movies_df["item_id"]==302]

,title,item_id
301,L.A. Confidential (1997),302


In [27]:
u_m_df=pd.merge(df,movies_df,on="item_id",how="left")
u_m_df

,user_id,item_id,rating,title
0,196,242,3,Kolya (1996)
1,186,302,3,L.A. Confidential (1997)
2,22,377,1,Heavyweights (1994)
3,244,51,2,Legends of the Fall (1994)
4,166,346,1,Jackie Brown (1997)
...,...,...,...,...
99995,880,476,3,"First Wives Club, The (1996)"
99996,716,204,5,Back to the Future (1985)
99997,276,1090,1,Sliver (1993)
99998,13,225,2,101 Dalmatians (1996)


In [31]:
reviews=u_m_df.groupby("title").size().reset_index().rename(columns={0:"n_reviews"})

In [32]:
reviews

,title,n_reviews
0,'Til There Was You (1997),9
1,1-900 (1994),5
2,101 Dalmatians (1996),109
3,12 Angry Men (1957),125
4,187 (1997),41
...,...,...
1659,Young Guns II (1990),44
1660,"Young Poisoner's Handbook, The (1995)",41
1661,Zeus and Roxanne (1997),6
1662,unknown,9


In [51]:
df2=u_m_df.merge(reviews,on="title",how="right")

In [55]:
df2[df2["title"]=='Reckless (1995)']

,user_id,item_id,rating,title,n_reviews
72469,280,1479,3,Reckless (1995),8
72470,311,1479,3,Reckless (1995),8
72471,269,1479,2,Reckless (1995),8
72472,405,1479,1,Reckless (1995),8
72473,450,1479,3,Reckless (1995),8
72474,655,1479,2,Reckless (1995),8
72475,707,1479,5,Reckless (1995),8
72476,846,1479,3,Reckless (1995),8


In [69]:
df2=df2[df2["n_reviews"]>=50]

In [70]:
movie_mat=df2.pivot_table(columns="user_id",index="title",values="rating").fillna(0)

In [71]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_mat.values)

In [77]:
movie_mat.head(3)

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2 Days in the Valley (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [67]:
from sklearn.neighbors import NearestNeighbors

In [74]:
model_knn=NearestNeighbors(n_neighbors=10,metric="cosine",algorithm="brute")

In [75]:
model_knn.fit(movie_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [96]:
movie_mat.loc[["Crimson Tide (1995)"]]

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Crimson Tide (1995),3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0


In [108]:
distances, indices=model_knn.kneighbors(movie_mat.loc[["Crimson Tide (1995)"]],n_neighbors=10)

In [109]:
distances.flatten()

array([0.        , 0.41400628, 0.42749614, 0.42908448, 0.43623728,
       0.4375568 , 0.43835664, 0.4426193 , 0.44322565, 0.4479048 ])

In [110]:

for i in range(len(indices.flatten())):
  print("{0}    {1}".format(i+1,movie_mat.index[indices.flatten()[i]]))
  

1    Crimson Tide (1995)
2    Forrest Gump (1994)
3    Braveheart (1995)
4    Firm, The (1993)
5    Raiders of the Lost Ark (1981)
6    True Lies (1994)
7    Hunt for Red October, The (1990)
8    Fugitive, The (1993)
9    Terminator 2: Judgment Day (1991)
10    Back to the Future (1985)
